# 5つの決定基準と分析手法

## マキシックス基準の実装

最大利得を最大にするのがマキシマックス基準

In [69]:
import pandas as pd
import numpy as np
from calc_payiff_table import calc_payoff_table

In [11]:
payoff = calc_payoff_table(100, 600, 0.2, 5000, 10000, 5000)
payoff

,0台,1台,2台
好調,-100.0,300.0,700.0
不調,-100.0,300.0,-300.0


In [12]:
# 選択肢ごとの最大利得
payoff.max()

0台   -100.0
1台    300.0
2台    700.0
dtype: float64

In [13]:
# 最大利得の最大
payoff.max().max()

700.0

In [14]:
payoff.max() == payoff.max().max()

0台    False
1台    False
2台     True
dtype: bool

In [16]:
# 最大値と等しい利得要素の取得
payoff.max()[payoff.max() == payoff.max().max()]

2台    700.0
dtype: float64

In [19]:
#　最大値と等しい利得要素インデックスの取得
list(payoff.max()[payoff.max() == payoff.max().max()].index)

['2台']

In [22]:
def argmax_list(series):
    return (list(series[series == series.max()].index))

In [23]:
def argmin_list(series):
    return (list(series[series == series.min()].index))

In [31]:
print("Maximax:", argmax_list(payoff.max()))

Maximax: ['2台']


## マキシミン基準

最小利得を最大にするのがマキシミン基準

In [30]:
# 選択肢ごとの最小利得
payoff.min()

0台   -100.0
1台    300.0
2台   -300.0
dtype: float64

In [28]:
print("Maximin:", argmax_list(payoff.min()))

Maximin: ['1台']


## ハーヴィッツの基準

最大利得（楽観的）と最小利得（悲観的）を楽観係数で重み付けした合計を計算のがハーヴィッツの基準

In [33]:
# alpha=0.6とした場合のハーヴィッツの基準
hurwicz = payoff.max() * 0.6 + payoff.min() * (1 - 0.6)
hurwicz

0台   -100.0
1台    300.0
2台    300.0
dtype: float64

In [34]:
argmax_list(hurwicz)

['1台', '2台']

楽観係数を0.6としたハーヴィッツの基準によると、１台稼働と２台稼働が同等に好ましいという結果になる

In [36]:
def hurwicz(payoff_table, alpha):
    hurwicz = payoff_table.max() * alpha + payoff_table.min() * (1 - alpha)
    return argmax_list(hurwicz)

In [48]:
print('Hurwicz:', hurwicz(payoff, 0.6))

Hurwicz: ['1台', '2台']


In [67]:
for i in range(1,11):
    cnt = i/10
    print("楽観係数: " + str(cnt) + ' Hurwicz:', hurwicz(payoff, cnt))

楽観係数: 0.1 Hurwicz: ['1台']
楽観係数: 0.2 Hurwicz: ['1台']
楽観係数: 0.3 Hurwicz: ['1台']
楽観係数: 0.4 Hurwicz: ['1台']
楽観係数: 0.5 Hurwicz: ['1台']
楽観係数: 0.6 Hurwicz: ['1台', '2台']
楽観係数: 0.7 Hurwicz: ['2台']
楽観係数: 0.8 Hurwicz: ['2台']
楽観係数: 0.9 Hurwicz: ['2台']
楽観係数: 1.0 Hurwicz: ['2台']


楽観係数が0.6以下では景気の先行きに対して悲観的になるため、１台稼働が選ばれるが、
0.6以上だと景気が良くなるという楽観視をすることが計算より求められた

## ミニマックスリグレット基準

リグレットとは状態ごとに見た最大利得から実際の利得を差し引いたもの（≒各状態の最大値ー実現値）

自然の状態ごとに見た最大利得と比較した差額（機会損失）を最小とする選択肢を採用するのがミニマックスリグレット基準

In [70]:
#　自然状態ごとに見た最大利得
best_df = pd.concat([payoff.max(axis=1)] * payoff.shape[1], axis=1)
best_df.columns = payoff.columns
print(best_df)

        0台    1台    2台
好調  700.0  700.0  700.0
不調  300.0  300.0  300.0


好調の時は700万円で、不調の時が300万円が最大利得である

リグレットとは自然の状態ごとに見た最大利得から実際の利得を差し引いたもの

In [72]:
# リグレット
regret_df = best_df - payoff
print(regret_df)

        0台    1台    2台
好調  800.0  400.0    0.0
不調  400.0    0.0  600.0


In [73]:
#　各々の選択肢におけるリグレットの最大値
regret_df.max()

0台    800.0
1台    400.0
2台    600.0
dtype: float64

リグレットの最大値が最小になるような選択を採択するのがミニマックスリグレット基準なので

In [75]:
argmin_list(regret_df.max())

['1台']

In [94]:
def minimax_regret(payoff_table):
    best_df = pd.concat([payoff_table.max(axis=1)] * payoff_table.shape[1], axis=1)
    best_df.columns = payoff_table.columns
    # リグレット
    regret_df = best_df - payoff_table # リグレットとは自然の状態ごとに見た最大利得から実際の利得を差し引いたもの

    # リグレットの最大値が最小になるような選択を採択するのがミニマックスリグレット基準
    return(argmin_list(regret_df.max()))

## ラプラス基準
平均利得が最大とするのがラプラス基準

In [95]:
#　選択肢ごとの平均利得
payoff.mean()

0台   -100.0
1台    300.0
2台    200.0
dtype: float64

In [96]:
print('Laplace:', argmax_list(payoff.mean()))

Laplace: ['1台']


## 感度分析
モデルの前提になった数値の変化が、意思決定の結果にどれほど影響を与えるかを調べる分析

ミニマックスリグレット基準を使って決定する際に、機械１台の稼働コストの推定誤差がもたらす影響を算出する

In [108]:
#　# 機械１台の稼働コストを25万円増加
payoff_1 = calc_payoff_table(fixed_cost=100, run_cost=600, sale_price=0.2,
                                machine_ability=5000, demand_boom=10000, demand_slump=5000)
print(payoff_1)
print('Minimax regret', minimax_regret(payoff_1))

        0台    1台    2台
好調 -100.0  300.0  700.0
不調 -100.0  300.0 -300.0
Minimax regret ['1台']


In [109]:
#　# 機械１台の稼働コストを25万円増加
payoff_2 = calc_payoff_table(fixed_cost=100, run_cost=625, sale_price=0.2,
                                machine_ability=5000, demand_boom=10000, demand_slump=5000)
print(payoff_2)
print('Minimax regret', minimax_regret(payoff_2))

        0台    1台    2台
好調 -100.0  275.0  650.0
不調 -100.0  275.0 -350.0
Minimax regret ['1台']


In [110]:
#　# 機械１台の稼働コストを25万円減少
payoff_3 = calc_payoff_table(fixed_cost=100, run_cost=575, sale_price=0.2,
                                machine_ability=5000, demand_boom=10000, demand_slump=5000)
print(payoff_3)
print('Minimax regret', minimax_regret(payoff_3))

        0台    1台    2台
好調 -100.0  325.0  750.0
不調 -100.0  325.0 -250.0
Minimax regret ['1台']


In [117]:
# 機械１台の稼働コストを100万円減少
payoff_4 = calc_payoff_table(fixed_cost=100, run_cost=500, sale_price=0.2,
                                machine_ability=5000, demand_boom=10000, demand_slump=5000)
print(payoff_4)
print('Minimax regret', minimax_regret(payoff_4))

        0台    1台    2台
好調 -100.0  400.0  900.0
不調 -100.0  400.0 -100.0
Minimax regret ['1台', '2台']


In [116]:
# 機械１代の稼働コストを125万円減少
payoff_5 = calc_payoff_table(fixed_cost=100, run_cost=475, sale_price=0.2,
                                machine_ability=5000, demand_boom=10000, demand_slump=5000)
print(payoff_5)
print('Minimax regret', minimax_regret(payoff_5))

        0台    1台    2台
好調 -100.0  425.0  950.0
不調 -100.0  425.0  -50.0
Minimax regret ['2台']


機械１台の稼働コストを初期値600万から25万ずつ増減させた際に、
100万減少させた場合は１台と２台が好ましく、100万以下は２台稼働だが、それ以外は１台が好ましい